# DEAP fuzzy chromosome experimentation
Experimenting with creating a chromosome to represent a fuzzy rule.

In [1]:
from deap import creator, base, algorithms, gp, tools
import random
import matplotlib.pyplot as plt
import seaborn as sns

import skfuzzy as fuzz
from skfuzzy import control as ctrl
from skfuzzy.control import Rule 
from skfuzzy.control.term import Term
import random
import operator
import numpy as np


# Define the primitive set for a fuzzy rule

In [2]:
def noop(x): return x

def makePrimitiveSet(antecendents, consequents):
    class make_consequents:
        def __init__(self):
            self.cons_terms = []
            for cons in consequents:
                for name in cons.terms.keys():
                    self.cons_terms.append(f"{cons.label}['{name}']")
            self.value = random.choice(self.cons_terms)
        def __repr__(self):
            return f"[{self.value}]"
    
    pset = gp.PrimitiveSetTyped("Rule", [], Rule)
    
    for ant in antecendents:
        pset.context[ant.label] = ant
        for name, term in ant.terms.items(): 
            pset.addTerminal(term, Term, f"{ant.label}['{name}']")

    for cons in consequents:
        pset.context[cons.label] = cons
            
    pset.addEphemeralConstant("consequents", make_consequents, list)
    pset.addPrimitive(Rule, [Term, list], Rule)
    pset.addPrimitive(operator.and_, [Term, Term], Term)
    pset.addPrimitive(operator.or_, [Term, Term], Term)
    pset.addPrimitive(operator.invert, [Term], Term)
    pset.addPrimitive(noop, [list], list)
    
    return pset


# Create the Antecedents and Consequents

In [3]:
petal_length = ctrl.Antecedent(np.linspace(1.0, 7.0, 11), 'petal_length')
petal_width = ctrl.Antecedent(np.linspace(0.0, 2.5, 11), 'petal_width')
petal_length.automf(names="short medium long".split())
petal_width.automf(names="narrow medium wide".split())

setosa = ctrl.Consequent(np.linspace(0, 1, 10), 'setosa', "som")
setosa['likely'] = fuzz.trimf(setosa.universe, (0., 1., 1.))

versicolor = ctrl.Consequent(np.linspace(0, 1, 10), 'versicolor', "som")
versicolor['likely'] = fuzz.trimf(versicolor.universe, (0., 1., 1.))

verginica = ctrl.Consequent(np.linspace(0, 1, 10), 'verginica', "som")
verginica['likely'] = fuzz.trimf(verginica.universe, (0., 1., 1.))

pset = makePrimitiveSet([petal_length, petal_width], [setosa, versicolor, verginica])


# Define constants 

In [4]:
POP_SIZE = 100
P_CROSSOVER = 0.9
P_MUTATION = 0.5
MAX_GENERATIONS = 30
HOF_SIZE = 10

MIN_TREE_HEIGHT = 1
MAX_TREE_HEIGHT = 3
MUT_MIN_TREE_HEIGHT = 0
MUT_MAX_TREE_HEIGHT = 3
LIMIT_TREE_HEIGHT = 17

# Define compile func, Individual class, select and mate operators etc

In [27]:
toolbox = base.Toolbox()

toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=MUT_MIN_TREE_HEIGHT, max_=MUT_MAX_TREE_HEIGHT)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=LIMIT_TREE_HEIGHT))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=LIMIT_TREE_HEIGHT))

def genRuleSet(pset, min_, max_, type_=None):
    len = random.randint(2, 5)
    return [gp.PrimitiveTree(gp.genFull(pset, min_, max_, type_)) for _ in range(len)]

creator.create("FitnessMin", base.Fitness, weights=(-1.,))
creator.create("Individual", list, fitness=creator.FitnessMin, pset=pset)
toolbox.register("expr", genRuleSet, pset=pset, min_=MIN_TREE_HEIGHT, max_=MAX_TREE_HEIGHT)
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

/home/dave/anaconda3/envs/eff/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/dave/anaconda3/envs/eff/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [28]:
i = toolbox.individualCreator()
# i
[toolbox.compile(x) for x in i]

[IF (NOT-petal_length[long]) AND (petal_length[short] AND petal_length[short]) THEN versicolor[likely]
 	AND aggregation function : fmin
 	OR aggregation function  : fmax,
 IF (petal_width[wide] OR petal_length[medium]) OR (NOT-petal_length[long]) THEN setosa[likely]
 	AND aggregation function : fmin
 	OR aggregation function  : fmax,
 IF petal_width[wide] THEN versicolor[likely]
 	AND aggregation function : fmin
 	OR aggregation function  : fmax,
 IF petal_width[medium] OR petal_width[wide] THEN verginica[likely]
 	AND aggregation function : fmin
 	OR aggregation function  : fmax,
 IF petal_length[short] OR petal_width[wide] THEN verginica[likely]
 	AND aggregation function : fmin
 	OR aggregation function  : fmax]

# Experimenting with the FuzzyClassifer class

In [2]:
%%time
from evofuzzy import fuzzyclassifier
from sklearn.datasets import load_iris
import pandas as pd

classifier = fuzzyclassifier.FuzzyClassifier()
data = load_iris()
cols = [c.replace(' ', '_').replace("_(cm)", "") for c in data.feature_names]
iris = pd.DataFrame(data.data, columns=cols)
y = pd.Series(data.target)
classes = {name: val for (name, val) in zip(data.target_names, range(3))}
classifier.fit(iris, y, classes)


gen	nevals	min	avg 
0  	100   	0  	0.84


AttributeError: 'PrimitiveTree' object has no attribute 'fitness'